In [1]:
import essentia
from essentia.standard import *
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [2]:
class AudioExtractor:

    def __init__(self,loop):
        self.loop = loop
        self.chopList = []
        self.fs = 44100
        self.w_onset = 'hann'
        self.mfccs = []
        self.w_mfcc = 'blackmanharris62'

    def __call__(self):
        return self.chopList

    def get_onsets(self):
        od = OnsetDetection(method='hfc')

        w = Windowing(type=self.w_onset)
        fft = FFT()  # this gives us a complex FFT
        c2p = CartesianToPolar()  # and this turns it into a pair (magnitude, phase)
        pool = essentia.Pool()

        # Computing onset detection functions.
        for frame in FrameGenerator(self.loop, frameSize=1024, hopSize=512):
            mag, phase, = c2p(fft(w(frame)))
            pool.add('features.hfc', od(mag, phase))

        onsets = Onsets()
        onsets_hfc_seconds = onsets(essentia.array([pool['features.hfc']]), [1])

        onsets_hfc_samples = onsets_hfc_seconds * self.fs #retun
        return onsets_hfc_samples, onsets_hfc_seconds


    def get_slices(self,onsets_hfc):
        onsets_hfc = onsets_hfc[0]  # onset stamps in samples
        for onset in range(len(onsets_hfc) - 1):
            self.chopList.append(self.loop[int(onsets_hfc[onset]):int((onsets_hfc[onset + 1]))])


    def extract_mfcc(self,slice):
        w = Windowing(type=self.w_mfcc)
        spectrum = Spectrum()
        mfcc = MFCC()
        mfcc_sin = []
        for frame in FrameGenerator(slice, frameSize=2048, hopSize=1024):
            mfcc_bands, mfcc_coeffs = mfcc(spectrum(w(frame)))
            mfcc_sin.append(mfcc_coeffs)
        mfcc_sin = essentia.array(mfcc_sin).T
        mfcc_sin = np.mean(mfcc_sin,axis=1)
        self.mfccs.append(mfcc_sin)


In [3]:
from essentia.standard import *
import numpy as np
from scipy.spatial import distance


class AudioManager:

    def __init__(self):
        self.d = {}
        self.d['main_loop'] = {}
        self.d['main_loop']['slices'] = {}

        self.d['slave_loop1'] = {}
        self.d['slave_loop2'] = {}
        self.d['slave_loop3'] = {}
        self.d['slave_loop1']['slices'] = {}
        self.d['slave_loop2']['slices'] = {}
        self.d['slave_loop3']['slices'] = {}

    def __call__(self):
        print(self.d)

    def add_slice(self,loop,slave_idx):
        extractor_loop = AudioExtractor(loop)
        onsets_hfc = extractor_loop.get_onsets()
        extractor_loop.get_slices(onsets_hfc)

        for nslice in range(len(extractor_loop.chopList)):
            if slave_idx==1:
                self.d['slave_loop1']['slices'][nslice] = {}
                self.d['slave_loop1']['slices'][nslice]['info'] = extractor_loop.chopList[nslice]

            if slave_idx==2:
                self.d['slave_loop2']['slices'][nslice] = {}
                self.d['slave_loop2']['slices'][nslice]['info'] = extractor_loop.chopList[nslice]

            if slave_idx==3:
                self.d['slave_loop3']['slices'][nslice] = {}
                self.d['slave_loop3']['slices'][nslice]['info'] = extractor_loop.chopList[nslice]

            if slave_idx==0:
                self.d['main_loop']['slices'][nslice] = {}
                self.d['main_loop']['slices'][nslice]['info'] = extractor_loop.chopList[nslice]

    def add_mfccs(self,loop,slave_idx):

        extractor_loop = AudioExtractor(loop)

        if slave_idx==1:

            for nslice in range(len(self.d['slave_loop1']['slices'])):
                extractor_loop.extract_mfcc(self.d['slave_loop1']['slices'][nslice]['info'])
                self.d['slave_loop1']['slices'][nslice]['mfccs'] = extractor_loop.mfccs[nslice]


        if slave_idx==2:
            for nslice in range(len(self.d['slave_loop2']['slices'])):
                extractor_loop.extract_mfcc(self.d['slave_loop2']['slices'][nslice]['info'])
                self.d['slave_loop2']['slices'][nslice]['mfccs'] = extractor_loop.mfccs[nslice]

        if slave_idx==3:
            for nslice in range(len(self.d['slave_loop3']['slices'])):
                extractor_loop.extract_mfcc(self.d['slave_loop3']['slices'][nslice]['info'])
                self.d['slave_loop3']['slices'][nslice]['mfccs'] = extractor_loop.mfccs[nslice]

        if slave_idx==0:
            for nslice in range(len(self.d['main_loop']['slices'])):
                extractor_loop.extract_mfcc(self.d['main_loop']['slices'][nslice]['info'])
                self.d['main_loop']['slices'][nslice]['mfccs'] = extractor_loop.mfccs[nslice]


    def similarity(self):

        for n in range(len(self.d['main_loop']['slices'])):
            min_dist = np.inf
            max_dist = 0
            idx = []

            #check the distances n slice --> to all slices in the slave loops
            for i in range(len(self.d['slave_loop1']['slices'])):
                new_dist = distance.euclidean(self.d['main_loop']['slices'][n]['mfccs'], self.d['slave_loop1']['slices'][i]['mfccs'])

                if new_dist > max_dist:
                    max_dist = new_dist

                if new_dist < min_dist:
                    min_dist = new_dist
                    #fisrt argument reference the slave loop and the second referene the index
                    idx = ['slave_loop1',i,min_dist/max_dist]


            for j in range(len(self.d['slave_loop2']['slices'])):
                new_dist = distance.euclidean(self.d['main_loop']['slices'][n]['mfccs'], self.d['slave_loop2']['slices'][j]['mfccs'])

                if new_dist > max_dist:
                    max_dist = new_dist

                if new_dist < min_dist:
                    min_dist = new_dist
                    idx = ['slave_loop2',j,min_dist/max_dist]

            for k in range(len(self.d['slave_loop3']['slices'])):
                new_dist = distance.euclidean(self.d['main_loop']['slices'][n]['mfccs'], self.d['slave_loop3']['slices'][k]['mfccs'])

                if new_dist > max_dist:
                    max_dist = new_dist

                if new_dist < min_dist:
                    min_dist = new_dist
                    idx = ['slave_loop3',k,min_dist/max_dist]


            #once we obtain the shortest distance we add the information to the dicctionary

            self.d['main_loop']['slices'][n]['similarity'] = idx


In [4]:
audio1 = MonoLoader(filename='../loops/audio1.wav')()
IPython.display.Audio(audio1,rate = 44100)

In [ ]:
extractor_loop = AudioExtractor(audio1)
onsets_hfc = extractor_loop.get_onsets()
extractor_loop.get_slices(onsets_hfc)
i = 2
slice0 = extractor_loop.chopList[i]
plt.plot(slice0)
plt.show()
print('From:', onsets_hfc[0][i],' to:', onsets_hfc[0][i+1])
IPython.display.Audio(slice0,rate = 44100)


In [ ]:
new_stamp = 17408
plt.plot(slice0)
plt.axvline(x=int(onsets_hfc[0][i+1])-int(onsets_hfc[0][i]), color='red')
plt.axvline(x=new_stamp - int(onsets_hfc[0][i]), color='green')
plt.show()

IPython.display.Audio(slice0,rate = 44100)
